# Create a manifest file

The *manifest.txt* file created here is intended to be used with [gdc-client](https://gdc.cancer.gov/access-data/gdc-data-transfer-tool)

Please refer to [gdc API](https://docs.gdc.cancer.gov/API/Users_Guide/Getting_Started/) for reference about query done here

In [1]:
%load_ext watermark
%watermark -v -m  -u -n -p pandas,requests,json -a Filippo_Valle -g -r -b -w

Author: Filippo_Valle

Last updated: Mon Dec 07 2020

Python implementation: CPython
Python version       : 3.8.6
IPython version      : 7.19.0

pandas  : 1.1.4
requests: 2.24.0
json    : 2.0.9

Compiler    : GCC 9.3.0
OS          : Linux
Release     : 5.4.39-linuxkit
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

Git hash: c31e7538af2d650caf235a80152a3cf4a3c2e784

Git repo: git@github.com:fvalle1/cancers

Git branch: develop_first_round

Watermark: 2.1.0



In [2]:
import requests
import json
import pandas as pd

In [3]:
# use file endopint
files_endpt = "https://api.gdc.cancer.gov/files"

# The 'fields' parameter is passed as a comma-separated string of single names.
fields = [
    "file_name",
    #"analysis.workflow_type",
    #"experimental_strategy",
    "cases.project.primary_site",
    "cases.project.disease_type",
    "cases.project.project_id",
    "cases.submitter_id",
    "cases.samples.portions.analytes.aliquots.submitter_id",
    "cases.demographic.vital_status",
    "cases.demographic.days_to_birth",
    "cases.demographic.days_to_death",
    "cases.demographic.gender",
    "cases.diagnoses.primary_diagnosis",
    "cases.diagnoses.tumor_stage",
    "cases.diagnoses.tumor_grade",
    "cases.diagnoses.last_known_disease_status",
    "cases.diagnoses.progression_or_recurrence"
    "cases.diagnoses.days_to_last_follow_up",
    "cases.diagnoses.age_at_diagnosis",
    "cases.diagnoses.morphology",
    "cases.diagnoses.tissue_or_organ_of_origin",
    "cases.exposures.years_smoked",
    "cases.exposures.cigarettes_per_day",
    "cases.samples.longest_dimension",
    "samples.portions.analytes.aliquots.submitter_id",
]

fields = ','.join(fields)

Filter gene expression data obtained with *RNA-Seq HTSeq* in txt format either from **BRCA** or **LUAD** and **LUSC**

Don't forget to select the correct filter for *cases.project.project_id*

In [4]:
# Tissue project filters
filters = {
    "op": "and",
    "content":[
        {
        "op": "in",
        "content":{
            "field": "files.data_type",
            "value": ["Gene Expression Quantification"]
            }
        },
        {
        "op": "in",
        "content":{
               "field": "files.analysis.workflow_type",
                "value": ["HTSeq - FPKM"]
                }
        },
        {
        "op": "in",
        "content":{
            "field": "files.data_format",
            "value": ["TXT"]
            }
        },
        {
        "op": "in",
        "content":{
            "field": "cases.project.project_id",
            "value": ['TCGA-LUAD', 'TCGA-LUSC'] #<-- uncomment this line for Lung
            #"value": ["TCGA-BRCA"] #<-- uncomment this line for Breast
            }
        }
        
    ]
}

In [5]:
# A POST is used, so the filter parameters can be passed directly as a Dict object.
##remove manifest to have file fields
params = {
   "return_type": "manifest",
    "filters": filters,
    "fields": fields,
    "format": "TSV",
    "size": "15000"
    }

In [6]:
# The parameters are passed to 'json' rather than 'params' in this case
response = requests.post(files_endpt, headers = {"Content-Type": "application/json"}, json = params)

#print(response.content.decode("utf-8"))

In [7]:
with open("manifest.txt","w") as manifest:
    manifest.write(response.content.decode("utf-8"))

# Files

Write a file with all metadata, useful in next analyses

In [8]:
params = {
    "filters": json.dumps(filters),
    "fields": "primary_site,cases.project.project_id,file_name,cases.demographic.vital_status,cases.demographic.gender,cases.diagnoses.age_at_diagnosis,cases.diagnoses.days_to_last_follow_up,cases.demographic.days_to_death,cases.demographic.days_to_birth,cases.submitter_id,samples.portions.analytes.aliquots.submitter_id,cases.diagnoses.last_known_disease_status,cases.diagnoses.tumor_stage,cases.exposures.years_smoked,cases.exposures.cigarettes_per_day,cases.samples.portions.analytes.aliquots.submitter_id",
    "format": "TSV",
    "size": "15000"
    }
response = requests.post(files_endpt, headers = {"Content-Type": "application/json"}, json = params)
with open("files.txt","w") as files:
    files.write(response.content.decode("utf-8"))

In [9]:
df_files = pd.read_csv("files.txt", sep='\t')
df_files.drop("id", axis=1, inplace=True)
df_files.set_index("file_name", inplace=True)
df_files.head(3)

,cases.0.demographic.days_to_birth,cases.0.demographic.days_to_death,cases.0.demographic.gender,cases.0.demographic.vital_status,cases.0.diagnoses.0.age_at_diagnosis,cases.0.diagnoses.0.days_to_last_follow_up,cases.0.diagnoses.0.last_known_disease_status,cases.0.diagnoses.0.tumor_stage,cases.0.exposures.0.cigarettes_per_day,cases.0.exposures.0.years_smoked,cases.0.project.project_id,cases.0.samples.0.portions.0.analytes.0.aliquots.0.submitter_id,cases.0.submitter_id
file_name,,,,,,,,,,,,,
e95d6e6b-6aaf-4003-9dc6-4b464a9d1725.FPKM.txt.gz,NaN,NaN,male,Alive,NaN,2109.0,not reported,stage ia,NaN,NaN,TCGA-LUAD,TCGA-55-6980-11A-01R-1949-07,TCGA-55-6980
96a02b1f-421c-40ee-8551-2b2f18709bcf.FPKM.txt.gz,-26444.0,1288.0,female,Dead,26444.0,NaN,not reported,stage ib,2.739726,NaN,TCGA-LUAD,TCGA-50-6590-01A-12R-1858-07,TCGA-50-6590
a7ac2239-f60b-40a2-b931-5aa9b3adc8d8.FPKM.txt.gz,-26289.0,NaN,male,Alive,26289.0,129.0,not reported,stage ia,2.191781,NaN,TCGA-LUAD,TCGA-69-8255-01A-11R-2287-07,TCGA-69-8255


In [10]:
df_files.columns

Index(['cases.0.demographic.days_to_birth',
       'cases.0.demographic.days_to_death', 'cases.0.demographic.gender',
       'cases.0.demographic.vital_status',
       'cases.0.diagnoses.0.age_at_diagnosis',
       'cases.0.diagnoses.0.days_to_last_follow_up',
       'cases.0.diagnoses.0.last_known_disease_status',
       'cases.0.diagnoses.0.tumor_stage',
       'cases.0.exposures.0.cigarettes_per_day',
       'cases.0.exposures.0.years_smoked', 'cases.0.project.project_id',
       'cases.0.samples.0.portions.0.analytes.0.aliquots.0.submitter_id',
       'cases.0.submitter_id'],
      dtype='object')

In [11]:
# save file to dat
df_files.to_csv("files_tcga.dat", header=True)